In [ ]:
!pip install torch
!pip install torchvision
!pip install mmcv
!pip install timm

In [6]:
import os, gc, torch

import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import xami_evaluate, get_iou_types
from models.load import get_trained_model
from utils.coco_eval import get_eval_params_dict
from data.datasets import collate_fn
from our_radiologist.load import get_anns
from utils.coco_utils import get_cocos, get_coco_api_from_dataset
from utils.eval import get_ap_ar
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.constants import XAMI_MIMIC_PATH, DEFAULT_REFLACX_LABEL_COLS
from utils.constants import full_iou_thrs, iou_thrs_5to95
from data.load  import seed_worker, get_dataloader_g
from tqdm import tqdm

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [7]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CPU]


In [8]:
from enum import Enum

class TrainedModels(Enum):
    # CXR_images="val_ar_0_6490_ap_0_2994_test_ar_0_6048_ap_0_3484_epoch76_09-01-2022 21-45-55_CXR_images"
    CXR_images_fixations= "val_ar_0_2079_ap_0_0753_test_ar_0_2067_ap_0_0909_epoch27_09-08-2022 13-14-00_CXR_images_fixations"

In [9]:
normal_iou_thrs = iou_thrs_5to95
all_range_iou_thrs = full_iou_thrs
score_thresholds = [0.05]

# Run evaluation.

In [11]:
for select_model in tqdm([
    # TrainedModels.CXR_images,
    TrainedModels.CXR_images_fixations,
]):

    for score_thrs in score_thresholds:

        model, train_info, _, _ = get_trained_model(
            select_model,
            DEFAULT_REFLACX_LABEL_COLS,
            device,
            rpn_nms_thresh=0.3,
            box_detections_per_img=10,
            box_nms_thresh=0.2,
            rpn_score_thresh=0.0,
            box_score_thresh=score_thrs,
        )

        model.eval()

        iou_types = get_iou_types(model, train_info.model_setup)

        model_setup = train_info.model_setup
        dataset_params_dict = {
        "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
        "bbox_to_mask": model_setup.use_mask,
        "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
        "with_fixation": True
        }


        detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict,
        )

        train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset, val_dataset, test_dataset, batch_size=4,
        )

        train_coco, val_coco, test_coco = get_cocos(
            train_dataloader, val_dataloader, test_dataloader
        )

        normal_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=normal_iou_thrs,
        )

        all_range_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=all_range_iou_thrs,
        )

        all_cat_ids = [None] + [
            detect_eval_dataset.disease_to_idx(d)
            for d in detect_eval_dataset.labels_cols
        ]

        for cat_id in all_cat_ids:
            cat_ids = (
                [
                    detect_eval_dataset.disease_to_idx(d)
                    for d in detect_eval_dataset.labels_cols
                ]
                if cat_id is None
                else [cat_id]
            )

            if not (cat_ids is None):
                all_range_eval_params_dict["bbox"].catIds = cat_ids
                all_range_eval_params_dict["segm"].catIds = cat_ids

            train_evaluator, _ = xami_evaluate(
                model,
                train_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=train_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            test_evaluator, _ = xami_evaluate(
                model,
                test_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            val_evaluator, _ = xami_evaluate(
                model,
                val_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=val_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            if cat_id is None:
                disease_str = "all"
            else:
                disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

            save_iou_results(
                train_evaluator,
                f"train_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                test_evaluator,
                f"test_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                val_evaluator,
                f"val_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )

            train_ap_ar = get_ap_ar(
                train_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            test_ap_ar = get_ap_ar(
                test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )
            val_ap_ar = get_ap_ar(
                val_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            df = pd.DataFrame(
                [
                    {
                        "dataset": "train",
                        f"AP@[IoBB = 0.50]": train_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": train_ap_ar["ar"],
                    },
                    {
                        "dataset": "test",
                        f"AP@[IoBB = 0.50]": test_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": test_ap_ar["ar"],
                    },
                    {
                        "dataset": "val",
                        f"AP@[IoBB = 0.50]": val_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": val_ap_ar["ar"],
                    },
                ]
            )

            df.to_csv(
                os.path.join(
                    "eval_results",
                    f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv",
                )
            )
            print_title(disease_str)
            print(df)


  0%|          | 0/1 [00:00<?, ?it/s]

Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3
CXR_images_fixations will use mask, [64] layers.
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Evaluation:  [  0/531]  eta: 0:30:54  loss: 0.4067 (0.4067)  loss_classifier: 0.0113 (0.0113)  loss_box_reg: 0.0056 (0.0056)  loss_mask: 0.3600 (0.3600)  loss_objectness: 0.0274 (0.0274)  loss_rpn_box_reg: 0.0024 (0.0024)  model_time: 1.4950 (1.4950)  evaluator_time: 0.3112 (0.3112)  time: 3.4917  data: 1.6855
Evaluation:  [100/531]  eta: 0:23:53  loss: 0.4774 (0.4382)  loss_classifier: 0.0117 (0.0129)  loss_box_reg: 0.0099 (0.0080)  loss_mask: 0.4071 (0.3892)  loss_objectness: 0.0236 (0.0247)  loss_rpn_box_reg: 0.0039 (0.0033)  model_time: 1.3883 (1.4438)  evaluator_time: 0.1785 (0.2334)  time: 3.2821  data: 1.5992
Evaluation:  [200/531]  eta: 0:18:14  loss: 0

100%|██████████| 1/1 [5:22:27<00:00, 19347.04s/it]

  dataset  AP@[IoBB = 0.50]  AR@[IoBB = 0.50]
0   train          0.017616          0.031630
1    test          0.021122          0.027397
2     val          0.000000          0.000000
